In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import cv2
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

In [ ]:
import ali

class MyConfig(Config):
    NUM_CLASSES = 6
    NAME = 'ali'

config = MyConfig()
config.display()

In [ ]:
DATA_DIR = "./jinnan2_round1_train_20190222/"

dataset_train = ali.AliDataset()
dataset_train.load_ali(DATA_DIR, 'train2019')
# Must call before using the dataset
dataset_train.prepare()

print("Train Image Count: {}".format(len(dataset_train.image_ids)))
print("Train Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

In [ ]:
NUM_EPOCHS = 1

# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")
model.train(dataset_train, dataset_train, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
model.train(dataset_train, dataset_train, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")